In [1]:
import requests as rq
import os
from tqdm import tqdm
import pprint
import json
from IPython.display import HTML, display

In [2]:
datapath = os.path.join("..","PycharmProjects","nlp","data", "ustawy")

In [3]:
ES_URL = "http://localhost:9200"
INDEX_NAME = "legislative-texts"
ANALYZER_SETUP = {
  "settings": {
    "analysis": {
      "analyzer": {
        "polish_legislative_analyzer": {
          "type": "custom", 
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "synonym",
            "morfologik_stem"
          ]
        }
      },
      "filter": {
        "synonym": {
            "type": "synonym",
            "lenient": True,
            "synonyms": [
              "kc => kodeks cywilny",
              "kk => kodeks karny",
              "kpc => kodeks postępowania cywilnego",
              "kpk => kodeks postępowania karnego"
            ]
          }
        }
    }
  },
  "mappings": {
    "properties": {
      "content": {
        "type": "text",
        "analyzer": "polish_legislative_analyzer"
      }
    }
  }
}

def display_repsonse(rsp):
    pprint.pprint(json.loads(rsp.content.decode()))
    
def get_doc_id(filename):
    return filename.split(".")[0]

def load_file_to_index(index, file_id, content, quiet=True):
    rsp = rq.put("{ES_URL}/{INDEX}/_doc/{FILE_ID}".format(ES_URL=ES_URL, 
                                                         INDEX=index,
                                                         FILE_ID=file_id), 
                 json={'content':content})
    if not quiet:
        display_repsonse(rsp)
    return rsp

def set_up_analyzer(index, analyzer_setup, quiet=True):
    rsp = rq.put("{ES_URL}/{INDEX}".format(ES_URL=ES_URL, 
                                           INDEX=index),
                json=analyzer_setup)
    if not quiet:
        display_repsonse(rsp)
    return rsp

def delete_index(index):
    rq.delete("{ES_URL}/{INDEX}".format(ES_URL=ES_URL, INDEX=index))

In [4]:
analyzer_response = set_up_analyzer(INDEX_NAME, ANALYZER_SETUP)

In [6]:
display_repsonse(analyzer_response)
response = rq.get("{ES_URL}/{INDEX}/_mapping".format(ES_URL=ES_URL, INDEX=INDEX_NAME))
display_repsonse(response)

{'acknowledged': True,
 'index': 'legislative-texts',
 'shards_acknowledged': True}
{'legislative-texts': {'mappings': {'properties': {'content': {'analyzer': 'polish_legislative_analyzer',
                                                               'type': 'text'}}}}}


In [7]:
load_text_responses = []
for doc_id, filename in enumerate(tqdm(os.listdir(datapath))):
    with open(os.path.join(datapath, filename)) as current_file:
        content = current_file.read()
        out = load_file_to_index(INDEX_NAME, doc_id, content, quiet=True)
        load_text_responses.append(out)
        
refresh_response = rq.post("{ES_URL}/{INDEX}/_refresh".format(ES_URL=ES_URL, INDEX=INDEX_NAME))

100%|██████████| 1180/1180 [01:01<00:00, 19.28it/s]


In [8]:
display_repsonse(load_text_responses[0])

{'_id': '0',
 '_index': 'legislative-texts',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': '_doc',
 '_version': 1,
 'result': 'created'}


In [9]:
query = {
  "query": {
    "match": {
      "content": "ustawa"
    }
  }
}
response = rq.get("{ES_URL}/{INDEX}/_count".format(ES_URL=ES_URL, INDEX=INDEX_NAME), json=query)
display_repsonse(response)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 1179}


In [10]:
query = {
    "query": {
        "intervals" : {
            "content" : {
                "all_of": {
                    "ordered": True,
                    "max_gaps": 0,
                    "intervals": [
                        {
                            "match": {
                                "query": "kodeks",

                            }
                        },
                        {
                            "match": {
                                "query": "postępowania",
                            }
                        },
                        {
                            "match": {
                                "query": "cywilnego",
                            }
                        },
                    ]
                }
            }
        }
    }
}
response = rq.get("{ES_URL}/{INDEX}/_doc/_count".format(ES_URL=ES_URL, INDEX=INDEX_NAME), json=query)
display_repsonse(response)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 100}


In [11]:
query = {
    "query": {
        "intervals" : {
            "content" : {
                "all_of": {
                    "ordered": True,
                    "max_gaps":2,
                    "intervals": [
                        {
                            "match": {
                                "query": "wchodzi",
                            }
                        },
                        {
                            "match": {
                                "query": "w",
                            }
                        },
                        {
                            "match": {
                                "query": "życie",
                            }
                        }                         
                    ]
                }
            }
        }
    }
}
response = rq.get("{ES_URL}/{INDEX}/_doc/_count".format(ES_URL=ES_URL, INDEX=INDEX_NAME), json=query)
display_repsonse(response)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 1175}


In [12]:
query = {
  "query": {
    "match": {
      "content": "konstytucja"
    }
  },
  "highlight": {
    "number_of_fragments": 3,
      "fragment_size": 50,
    "fields": {
      "content": {}
    }
  }
}
response = rq.get("{ES_URL}/{INDEX}/_search?size=10".format(ES_URL=ES_URL, INDEX=INDEX_NAME), json=query)
rsp = json.loads(response.content.decode())

In [13]:
score_id = [[h['_score'], h['_id'], h['highlight']['content']] for h in rsp['hits']['hits']]
display(HTML(
   '<table><tr><th>Score</th><th>Id</th><th>Higlights</th></tr><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(el) if type(el) != list else "</br>".join(el) for el in row)) for row in score_id)
       )
))

Score,Id,Higlights
6.8715763,1095,"Do zgłoszenia projektu Konstytucji załącza się wykazZasady, na których opierać się ma Konstytucja mogąProjekty Konstytucji wniesione na podstawie i w trybie"
6.6677475,69,"zgody, o której mowa w art. 89 ust. 1 i art. 90 Konstytucjimiędzynarodowej, o którym mowa w art. 89 ust. 2 Konstytucjizgodą, o której mowa w art. 89 ust. 1 i art. 90 Konstytucji"
6.63778,1025,Z wnioskiem o stwierdzenie zgodności z KonstytucjąWnioski o stwierdzenie zgodności z Konstytucją celówCiężar udowodnienia niezgodności z Konstytucją spoczywa
6.63198,106,"Za naruszenie Konstytucji lub ustawy, w związku z zajmowanymW zakresie określonym w art. 107 Konstytucji odpowiedzialnośćprzed Trybunałem Stanu za naruszenie Konstytucji"
6.2563562,494,"zgodności ustaw i umów międzynarodowych z Konstytucjąprzez centralne organy państwowe, z Konstytucjąustawami, 4) stwierdzenie zgodności z Konstytucją"
6.06512,120,"pokoju, 2) referendum zatwierdzającego zmianę Konstytucjiważności referendum zatwierdzającego zmianę Konstytucjiwyboru, powoływania lub odwoływania na określone w Konstytucji"
5.9354334,1005,kwietnia 1992 r. o trybie przygotowania i uchwalenia Konstytucjikwietnia 1992 r. o trybie przygotowania i uchwalenia KonstytucjiPrzyjęcie w referendum Konstytucji następuje wówczas
5.8562784,211,"przez obywateli, o której mowa w art. 118 ust. 2 KonstytucjiProjekt ustawy nie może dotyczyć spraw, dla których KonstytucjaMarszałka Sejmu powinien odpowiadać wymogom zawartym w Konstytucji"
5.473364,402,"zwana dalej ""Radą"", realizuje cele określone w KonstytucjiTrybunału Konstytucyjnego o zbadanie zgodności z Konstytucją"
5.473364,481,"Rzecznikiem, stoi na straży praw dziecka określonych w Konstytucjiobowiązków Rzecznika Praw Dziecka dochowam wierności Konstytucji"


In [387]:
#delete_index(INDEX_NAME)